¿Qué Es el Web Scraping?
El web scraping es un conjunto de prácticas utilizadas para extraer automáticamente — o «scrapear» — datos de la web.

El web scraping se refiere al proceso de extracción de contenidos y datos de sitios web mediante software.

Este script realiza web scraping de la página de un libro específico en el sitio "Books to Scrape".

Primero, hacemos una petición HTTP con `requests.get(url)` para obtener el contenido HTML de la página.

Luego, usamos `BeautifulSoup(response.text, 'html.parser')` para convertir ese HTML crudo en un objeto navegable.
Esto permite buscar elementos fácilmente usando métodos como `.find()`, `.select_one()`, etc.

Extraemos el título con `soup.find('h1').text`, el precio con `soup.select_one('.price_color').text`,  y lo limpiamos con `.replace()` para quitar caracteres no deseados (como "Â").

Para la disponibilidad, usamos `.select_one('.availability').text.strip()` para eliminar espacios en blanco.

La descripción del libro está justo después del div con id `product_description`, por eso usamos `.find_next_sibling('p')`.

Finalmente, las estrellas están codificadas como una clase CSS en la etiqueta <p>, como por ejemplo: `<p class="star-rating Three">.`

Al usar `tag['class'][1]`, obtenemos el texto que indica la cantidad de estrellas (por ejemplo, "Three").

In [ ]:
# Extraer la información de un solo libro

import requests
from bs4 import BeautifulSoup #Averiguar bien

# URL del libro
url = 'https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'

# Hacemos la petición
response = requests.get(url)

# BeautifulSoup(response.text, 'html.parser') transforma el HTML crudo en un objeto navegable.
soup = BeautifulSoup(response.text, 'html.parser')

# Obtenemos el título
title = soup.find('h1').text

# .text te devuelve el texto visible dentro de una etiqueta HTML.
# La gente usa .text sin saber qué devuelve la etiqueta. Si la etiqueta no tiene texto visible (ej: <img> o <meta>), .text te da vacío.

# Obtenemos precio
price = soup.select_one('.price_color').text
price = price.replace('Â', '')  # Limpia cualquier carácter raro que aparezca

# Obtenemos disponibilidad
availability = soup.select_one('.availability').text.strip()

# Descripción (viene en el siguiente <p> después del div con id 'product_description')
desc_tag = soup.find('div', id='product_description')
# Usamos `.find_next_sibling('p')`, que busca el **próximo hermano (sibling)** que sea una etiqueta <p>.
description = desc_tag.find_next_sibling('p').text if desc_tag else "Sin descripción"

# Las estrellas están codificadas en la clase CSS de un <p>
# Ese guion bajo es una convención para evitar conflicto. Internamente, BeautifulSoup ya sabe que class_ se refiere al atributo class del HTML
# Al hacer tag['class'], obtenemos una lista como ['star-rating', 'Three'], y el índice [1] nos da la cantidad de estrellas en texto.
star_tag = soup.find('p', class_='star-rating')
stars = star_tag['class'][1]

# Imprimimos
print(f"Título: {title}")
print(f"Precio: {price}")
print(f"Disponibilidad: {availability}")
print(f"Descripción: {description[:60]}...")
print(f"Estrellas: {stars}")


Título: A Light in the Attic
Precio: £51.77
Disponibilidad: In stock (22 available)
Descripción: It's hard to imagine a world without A Light in the Attic. T...
Estrellas: Three


Scrapeo de varias páginas

Que hacen:

1. requests.get(url) hace una petición HTTP a la URL y devuelve la respuesta con el HTML.

2. BeautifulSoup transforma el texto HTML en un objeto para buscar etiquetas fácilmente.

3. .find() busca la primera etiqueta que cumple la condición.

4. .select() busca todas las etiquetas que cumplen un selector CSS (devuelve lista).

5. .select_one() busca el primer elemento que cumple el selector CSS (devuelve un tag).

6. En las URLs, a veces hay rutas relativas con "../", hay que limpiarlas para crear URLs absolutas.

7. time.sleep(0.1) hace que el programa espere 0.1 segundos para no saturar el servidor y evitar bloqueos.

8. El bucle while True se usa para seguir scrapeando páginas hasta que no haya más páginas siguientes.

9. Las estrellas del libro están en la clase CSS 'star-rating X', donde X es la cantidad en texto (One, Two, Three, etc).

10. Guardamos los datos en una lista de diccionarios para luego exportarlos o procesarlos fácilmente.

In [10]:
# Extraer la información de una categoría

import requests # Para hacer peticiones HTTPS y obtener el contenido de las páginas
from bs4 import BeautifulSoup # Para parsear (analizar) el contenido de la página web
import time # Para pausar entre peticiones y no sobrecargar el servidor

base_url = 'https://books.toscrape.com/catalogue/'

category_url = 'https://books.toscrape.com/catalogue/category/books/mystery_3/index.html'

# Lista donde se guardaran los datos de todos los libros
all_books = []

# Función scrape_book, toma la url de un libro especifico y devuelve los datos scrapeados en forma de diccionario
def scrape_book(url):

  # Hacemos la petición
  try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()
  except requests.RequestException as e:
    print(f"❌ Error al acceder a {url}: {e}")
    return None

  # BeautifulSoup(response.text, 'html.parser') transforma el HTML crudo en un objeto navegable.
  soup = BeautifulSoup(response.text, 'html.parser')

  # Obtenemos el título
  title = soup.find('h1').text

  # Obtenemos precio
  price = soup.select_one('.price_color').text
  price = price.replace('Â', '')  # Limpia cualquier carácter raro que aparezca

  # Obtenemos disponibilidad
  availability = soup.select_one('.availability').text.strip()

  # Descripción (viene en el siguiente <p> después del div con id 'product_description')
  desc_tag = soup.find('div', id='product_description')
  # Usamos `.find_next_sibling('p')`, que busca el **próximo hermano (sibling)** que sea una etiqueta <p>.
  description = desc_tag.find_next_sibling('p').text if desc_tag else "Sin descripción"

  # Las estrellas están codificadas en la clase CSS de un <p>
  # Ese guion bajo es una convención para evitar conflicto. Internamente, BeautifulSoup ya sabe que class_ se refiere al atributo class del HTML
  # Al hacer tag['class'], obtenemos una lista como ['star-rating', 'Three'], y el índice [1] nos da la cantidad de estrellas en texto.
  star_tag = soup.find('p', class_='star-rating')
  stars = star_tag['class'][1]

  return {
    'Titulo': title,
    'Precio': price,
    'Disponibilidad': availability,
    'Estrellas': stars,
    'URL': url
  }

# Funcion scrape_category, toma la url de una categoria, recorre todas sus paginas y scrapea todos los libros que encuentra
def scrape_category(url):
  while True:
    print(f"Scrapeando pagina: {url}")
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Seleccionamos todos los libros en la pagina actual de esa categoría
    book_articles = soup.select('article.product_pod')

    for book in book_articles:

      # Obtenemos el href del libro
      relative_url = book.find('h3').find('a')['href']

      # relative_url trae la url completa, entonces la limpiamos para después concatenarla a base_url
      book_url = base_url + relative_url.replace('../../../', '')

      # Scrapeamos el libro
      book_data = scrape_book(book_url)

      # Guardamos el resultado
      all_books.append(book_data)

      # Pausa para evitar sobrecarga
      time.sleep(0.1)

    # Verificamos si hay un boton next para seguir scrapeando la sgte pagina
    # Usa un selector CSS (li.next > a) para encontrar el link de la siguiente página. Si existe: next_button va a tener un tag <a>.
    next_button = soup.select_one('li.next > a')
    if next_button:
      next_page_url = next_button['href']
      # rsplit(separador, cantidad) divide un string desde el final, y solo hace la cantidad de cortes que vos le indiques.
      url = url.rsplit('/', 1)[0] + '/' + next_page_url
    else:
      break

scrape_category(category_url)

print(f"Total de libros scrapeados: {len(all_books)}")
print(all_books)

Scrapeando pagina: https://books.toscrape.com/catalogue/category/books/mystery_3/index.html
Scrapeando pagina: https://books.toscrape.com/catalogue/category/books/mystery_3/page-2.html
Total de libros scrapeados: 32
[{'Titulo': 'Sharp Objects', 'Precio': '£47.82', 'Disponibilidad': 'In stock (20 available)', 'Estrellas': 'Four', 'URL': 'https://books.toscrape.com/catalogue/sharp-objects_997/index.html'}, {'Titulo': 'In a Dark, Dark Wood', 'Precio': '£19.63', 'Disponibilidad': 'In stock (18 available)', 'Estrellas': 'One', 'URL': 'https://books.toscrape.com/catalogue/in-a-dark-dark-wood_963/index.html'}, {'Titulo': 'The Past Never Ends', 'Precio': '£56.50', 'Disponibilidad': 'In stock (16 available)', 'Estrellas': 'Four', 'URL': 'https://books.toscrape.com/catalogue/the-past-never-ends_942/index.html'}, {'Titulo': 'A Murder in Time', 'Precio': '£16.64', 'Disponibilidad': 'In stock (16 available)', 'Estrellas': 'One', 'URL': 'https://books.toscrape.com/catalogue/a-murder-in-time_877/inde